In [1]:
from qiskit_optimization.applications import Knapsack
import pandas as pd

from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler

In [2]:
data = pd.read_csv('medium.csv')

In [3]:
data

,77,95
0,44,70
1,15,85
2,67,31
3,75,100
4,91,1
5,2,7
6,46,53
7,16,35
8,68,11
9,35,64


In [4]:
values = data.iloc[:,0]
weights = data.iloc[:,1]
max_weight = int(0.6 * sum(weights))

In [5]:
sum( weights)

483

In [6]:
print(f'values = {values}, weights={weights},max_weight={max_weight}')


values = 0     44
1     15
2     67
3     75
4     91
5      2
6     46
7     16
8     68
9     35
10    53
Name: 77, dtype: int64, weights=0      70
1      85
2      31
3     100
4       1
5       7
6      53
7      35
8      11
9      64
10     26
Name: 95, dtype: int64,max_weight=289


In [7]:
prob = Knapsack(values=values, weights=weights, max_weight=max_weight)
qp = prob.to_quadratic_program()
print(qp.prettyprint())

Problem name: Knapsack

Maximize
  44*x_0 + 15*x_1 + 53*x_10 + 67*x_2 + 75*x_3 + 91*x_4 + 2*x_5 + 46*x_6 + 16*x_7
  + 68*x_8 + 35*x_9

Subject to
  Linear constraints (1)
    70*x_0 + 85*x_1 + 26*x_10 + 31*x_2 + 100*x_3 + x_4 + 7*x_5 + 53*x_6 + 35*x_7
    + 11*x_8 + 64*x_9 <= 289  'c0'

  Binary variables (11)
    x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9 x_10



In [8]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [ ]:
# Numpy Eigensolver
meo = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
result = meo.solve(qp)
print(result.prettyprint())
print("\nsolution:", prob.interpret(result))

objective function value: 435.0
variable values: x_0=0.0, x_1=0.0, x_2=1.0, x_3=1.0, x_4=1.0, x_5=0.0, x_6=1.0, x_7=0.0, x_8=1.0, x_9=1.0, x_10=1.0
status: SUCCESS

solution: [2, 3, 4, 6, 8, 9, 10]


: 

In [ ]:
# QAOA
meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, sampler=Sampler(), optimizer=COBYLA()))
result = meo.solve(qp)
print(result.prettyprint())
print("\nsolution:", prob.interpret(result))
print("\ntime:", result.min_eigen_solver_result.optimizer_time)

Traceback (most recent call last):
  File "/Users/thiru/anaconda3/envs/qiskit_1.0/lib/python3.12/site-packages/scipy/optimize/_cobyla_py.py", line 281, in calcfc
capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
    f = sf.fun(x)
        ^^^^^^^^^
  File "/Users/thiru/anaconda3/envs/qiskit_1.0/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py", line 325, in fun
    self._update_fun()
  File "/Users/thiru/anaconda3/envs/qiskit_1.0/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py", line 294, in _update_fun
    fx = self._wrapped_fun(self.x)
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thiru/anaconda3/envs/qiskit_1.0/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py", line 20, in wrapped
    fx = fun(np.copy(x), *args)
         ^^^^^^^^^^^^^^^^^^^^^^


In [ ]:
from qiskit_optimization.converters import QuadraticProgramToQubo


In [ ]:
# the same knapsack problem instance as in the previous section
prob = Knapsack(values=values, weights=weights, max_weight=max_weight)
qp = prob.to_quadratic_program()
print(qp.prettyprint())

In [ ]:
# intermediate QUBO form of the optimization problem
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
print(qubo.prettyprint())

In [ ]:
# qubit Hamiltonian and offset
op, offset = qubo.to_ising()
print(f"num qubits: {op.num_qubits}, offset: {offset}\n")
print(op)